<a href="https://colab.research.google.com/github/gusanagy/DeepLabV3/blob/main/DeeplabV3_Autoceres_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#DeeplabV3+

* investigar collate function

* Atualizar no github
* Organizar codigo

* ajustar passo de treino
* debugar erros no formato batches incompatible


##Bibliotecas usadas

In [ ]:
import os
import numpy as np
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from glob import glob
from tqdm import tqdm_notebook as tqdm
from google.colab import drive

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

from albumentations import Compose, HorizontalFlip, ChannelShuffle, Resize, CoarseDropout, Rotate, Equalize, RandomShadow, CenterCrop, RandomBrightnessContrast, HorizontalFlip, CLAHE, Normalize
#from albumentations.augmentations.geometric.resize import Resize as
from albumentations.pytorch import ToTensorV2

import torch
from torch import optim
from torch import nn as nn

from torch.utils.data import Dataset, DataLoader
from torch.utils.data import SubsetRandomSampler

from torchvision import datasets
from torchvision.transforms import ToTensor, Resize
from torchvision.io import read_image
from torchvision.transforms import ConvertImageDtype
from torchvision import datasets, transforms
#from torchvision.transforms import Compose
from torchvision import transforms

## Montar Drive

In [ ]:

# montar Google Drive / mount Google drive
# será solicitadas as credenciais do usuario
# do gmail
drive.mount('/content/gdrive/',force_remount=True)
#ir para pasta conhecimento
path = '/content/gdrive/MyDrive/DeepLabV3Plus-Pytorch/'
#ir para pasta conhecimento
os.chdir(path)

#listar arquivos da pasta
!ls -A

Mounted at /content/gdrive/
dataset   .gitignore	       LICENSE	network      README.md	       utils
datasets  Heart.csv	       main.py	predict.py   requirements.txt
.git	  helper_functions.py  metrics	__pycache__  samples


### Setup Agnostic code

In [ ]:
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
# Setup device-agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

##Data Augmentation Functions

In [ ]:
def augment_data(images, masks, save_path, augment = True):
    H = 512
    W = 512

    for x, y in tqdm(zip(images, masks), total = len(images)):
        name = x.split("/")[-1].split(".")[0]

        x = cv2.imread(x, cv2.IMREAD_COLOR)
        y = cv2.imread(y, cv2.IMREAD_COLOR)

        """ Data Augmentation """
        if augment == True:
            aug = HorizontalFlip(p = 1.0) #Flip horizontal
            augmented = aug(image = x, mask = y)
            x1 = augmented["image"]
            y1 = augmented["mask"]

            x2 = cv2.cvtColor(x, cv2.COLOR_RGB2GRAY)
            y2 = y

            """
            aug = ChannelShuffle(p = 1) #Reorganiza aleatoriamente os canais da imagem RGB de entrada.
            augmented = aug(image = x, mask = y)
            x3 = augmented['image']
            y3 = augmented['mask']
            """

            aug = CoarseDropout(p = 1, min_holes = 3, max_holes = 10, max_height = 32, max_width = 32) #Dropa uma caixa de pixels 32x32 da imagem, pode simular possíveis falhas na linha de plantação
            augmented = aug(image = x, mask = y)
            x4 = augmented['image']
            y4 = augmented['mask']

            """
            aug = Rotate(limit = 45, p = 1.0) #Rotaciona a imagem de entrada
            augmented = aug(image = x, mask = y)
            x5 = augmented["image"]
            y5 = augmented["mask"]
            """

            aug = Equalize(p = 1.0) #Equaliza a imagem de entrada
            augmented = aug(image = x, mask = y)
            x5 = augmented["image"]
            y5 = augmented["mask"]

            """
            aug = RandomShadow(shadow_roi = (0, 0.5, 1, 1), num_shadows_lower = 1, num_shadows_upper = 1, shadow_dimension = 4, always_apply = False, p=1) #Cria sombras nas imagens
            augmented = aug(image = x, mask = y)
            x6 = augmented["image"]
            y6 = augmented["mask"]
            """

            aug = RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, brightness_by_max=True, always_apply=False, p=1.5) #Brilho aleatório
            augmented = aug(image = x, mask = y)
            x6 = augmented["image"]
            y6 = augmented["mask"]

            X = [x, x1, x2, x4, x5, x6]
            Y = [y, y1, y2, y4, y5, y6]

        else:
            X = [x]
            Y = [y]

        index = 0
        for i, m in zip(X, Y):
            try:
                aug = CenterCrop(H, W, p = 1.0)
                augmented = aug(image = i, mask = m)
                i = augmented["image"]
                m = augmented["mask"]

            except Exception as e:
                i = cv2.resize(i, (W, H))
                m = cv2.resize(m, (W, H))

            tmp_image_name = f"{name}_{index}.png"
            tmp_mask_name = f"{name}_{index}.png"

            image_path = os.path.join(save_path, "image", tmp_image_name)
            mask_path = os.path.join(save_path, "mask", tmp_mask_name)

            cv2.imwrite(image_path, i)
            cv2.imwrite(mask_path, m)

            index += 1

In [ ]:
""" Diretório """
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_data(path, split = 0.25):
    """ Carregamento das imagens e máscaras """
    X = sorted(glob(os.path.join(path, "/content/gdrive/MyDrive/DeepLabV3Plus-Pytorch/datasets/Dataset_Strawberry/images", "*.png")))
    Y = sorted(glob(os.path.join(path, "/content/gdrive/MyDrive/DeepLabV3Plus-Pytorch/datasets/Dataset_Strawberry/masks", "*.png")))

    split_size = int(len(X) * split)

    train_x, test_x = train_test_split(X, test_size = split_size, random_state = 42)
    train_y, test_y = train_test_split(Y, test_size = split_size, random_state = 42)

    return (train_x, train_y), (test_x, test_y)

## Data Augmentation Main

In [ ]:
# if __name__ == "__main__":
#     np.random.seed(42)

#     """ Carregamento do Dataset """
#     data_path = "datasets"

#     (train_x, train_y), (test_x, test_y) = load_data(data_path)

#     print(f"Train:\t {len(train_x)} - {len(train_y)}")
#     print(f"Test:\t {len(test_x)} - {len(test_y)}")

#     """ Criando os diretórios para o Data Augmentation """
#     create_dir("/dataset/new_data/train/image/")
#     create_dir("/dataset/new_data/train/mask/")
#     create_dir("/dataset/new_data/test/image/")
#     create_dir("/dataset/new_data/test/mask/")

#     augment_data(train_x, train_y, "/dataset/new_data/train/", augment = True)
#     augment_data(test_x, test_y, "/dataset/new_data/test/", augment = True)#Substitui por True

## Setup DataLoader

### Endereço para o dataset Augumentado com a lib augumentations

In [ ]:
#diretório das imagens de treino
TRAIN_IMG_DIR = "dataset/new_data/train/image/"
 #diretório das máscaras de treino
TRAIN_MASK_DIR = "dataset/new_data/train/mask/"
#diretório das imagens de teste
VAL_IMG_DIR = "dataset/new_data/test/image/"
#diretório das máscaras de teste
VAL_MASK_DIR = "dataset/new_data/test/mask/"

#Caso fosse utilizar o dataset sem augumentar
DIR_X = "datasets/Dataset_Strawberry/images/"
DIR_y = "datasets/Dataset_Strawberry/masks/"

print(f"""
          Dir of X / no aug:     {DIR_X}
          Dir of y / no aug:     {DIR_y}
          Dir train x:  {TRAIN_IMG_DIR}
          Dir train y:  {TRAIN_MASK_DIR}
          Dir test x:   {VAL_IMG_DIR}
          Dir test y:   {VAL_MASK_DIR}""")



          Dir of X / no aug:     datasets/Dataset_Strawberry/images/
          Dir of y / no aug:     datasets/Dataset_Strawberry/masks/
          Dir train x:  dataset/new_data/train/image/
          Dir train y:  dataset/new_data/train/mask/
          Dir test x:   dataset/new_data/test/image/
          Dir test y:   dataset/new_data/test/mask/


### Classe Custom Dataset

In [ ]:


def one_hot_encode(label, label_values):
    """
    Convert a segmentation image label array to one-hot format
    by replacing each pixel value with a vector of length num_classes
    # Arguments
        label: The 2D array segmentation image label
        label_values

    # Returns
        A 2D array with the same width and hieght as the input, but
        with a depth size of num_classes
    """
    semantic_map = []
    for colour in label_values:
        equality = np.equal(label, colour)
        class_map = np.all(equality, axis = -1)
        semantic_map.append(class_map)
    semantic_map = np.stack(semantic_map, axis=-1)

    return semantic_map

class AgroDataset(Dataset):
    def __init__(self, img_dir_x,img_dir_y, transform=None,transform_2=None):
        #self.img_labels = pd.read_csv(annotations_file)
        self.img_dir_x = img_dir_x
        self.img_dir_y = img_dir_y
        self.train_transform = transform
        self.train_transform_2 = transform_2
        self.images = os.listdir(img_dir_x)

    def __len__(self):
        return len(self.img_dir_x)

    def __getitem__(self, idx):
        img_path_x = os.path.join(self.img_dir_x,self.images[idx].replace(".jpg", ".png"))
        img_path_y = os.path.join(self.img_dir_y,self.images[idx].replace(".jpg", ".png"))
        image_x =  np.array(Image.open(img_path_x).convert("RGB"))#Modifiqui para nao carretgar imagens rgb
        label_mask_y = np.array(Image.open(img_path_y).convert("L"))

        #label_mask_y = label_mask_y.repeat(3, 1, 1)
        #image_x = read_image(img_path_x)#.numpy().dtype(np.int32)
        #label_mask_y = read_image(img_path_y)#.numpy().dtype(np.int32)
        #label_mask_y = Image.open(img_path_y)
        #image_x = Image.open(img_path_x)
        #label = os.path.basename(X_train[idx])
        #label_mask_y[label_mask_y ==255.0] = 1.0

        #convert_to_binary = lambda x: 1.0 if x == 255 else 0.0
        #resultado = list(map(convert_to_binary, label_mask_y))

        if self.train_transform is not None:
          image_x = self.train_transform(image_x)
          label_mask_y = self.train_transform_2(label_mask_y)
        return image_x, label_mask_y
    # classmethod
    # def decode_target(cls, mask):
    #     """decode semantic mask to RGB image"""
    #     return cls.cmap[mask]


###Instancia a Classe e carrega os dados do Dataset

In [ ]:
IMAGE_HEIGHT = 160
IMAGE_WIDTH = 240
BATCH = 8
WORKERS = 4


# Definir transformações para o DataLoader
train_transform = transforms.Compose(
                              [
                              transforms.ToPILImage(),
                              transforms.Resize((255,255)),
                              transforms.ToTensor(),
                              #transforms.Lambda(lambda x: torch.cat([x, x, x], dim=0)),
                              #transforms.Lambda(lambda x: 1.0 if x == 255.0 else x / 255.0),
                              #transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
                              #transforms.Normalize((0.485), (0.229)),
                              transforms.ConvertImageDtype(torch.float)
                              ]
)
train_transform_2 = transforms.Compose(
                              [
                              transforms.ToPILImage(),
                              transforms.Resize((255,255)),
                              transforms.ToTensor(),
                              #transforms.Lambda(lambda x: torch.cat([x, x, x], dim=0)),
                              #transforms.Lambda(lambda x: 1.0 if x == 255.0 else x / 255.0),
                              #transforms.Normalize((0.485), (0.229)),
                              transforms.ConvertImageDtype(torch.float)
                              ]
)


# train_transform = Compose(
#     [
#         Resize((IMAGE_HEIGHT,IMAGE_WIDTH)),
#         Rotate(limit=35, p=0.3),                        #Rotaciona em até 35 graus
#         HorizontalFlip(p=1.0),                          #Espelha horizontalmente com probabilidade de 0,5
#         CLAHE(p=0.2),                                   #ajuda a melhorar a visibilidade de detalhes em regiões escuras ou claras da imagem.
#         Normalize(                   #normaliza os valores de pixel da imagem para ter média 0 e desvio padrão 1
#             mean=[0.5, 0.5, 0.5],
#             std=[0.5, 0.5, 0.5],
#             max_pixel_value=255.0,
#         ),
#         ToTensorV2(),     #converte a imagem para o formato tensor
#     ],
# )
# val_transforms = Compose(
#         [
#             Resize((IMAGE_HEIGHT,IMAGE_WIDTH)),   #redimensiona a imagem
#             Normalize(
#                 mean=[0.5, 0.5, 0.5],
#                 std=[0.5, 0.5, 0.5],
#                 max_pixel_value=255.0,
#             ),
#             ToTensorV2(),
#         ],
#     )

data_train = AgroDataset( img_dir_x = TRAIN_IMG_DIR,
                          img_dir_y = TRAIN_MASK_DIR,
                          transform = train_transform,
                          transform_2=train_transform_2
                          #transform = ConvertImageDtype(torch.int),
                          #target_transform = ConvertImageDtype(torch.int)
                              )
data_test = AgroDataset(  img_dir_x = VAL_IMG_DIR,
                          img_dir_y = VAL_IMG_DIR,
                          transform = train_transform,
                          transform_2=train_transform_2
                          #transform = ConvertImageDtype(torch.int),
                          #target_transform = ConvertImageDtype(torch.int)
                              )

train_dataloader = DataLoader(
                              data_train,
                              shuffle=True,
                              batch_size=BATCH,
                              num_workers=WORKERS,
                              drop_last=True
                              )

test_dataloader  = DataLoader(
                              data_test,
                              shuffle=True,
                              batch_size=BATCH,
                              num_workers=WORKERS
                              )

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Fazer um preprocessing que seriva para os dois??

Carregar os dados de fomra diferente?

Implementar on-hot encode?


In [ ]:
#batch_images, batch_labels = next(iter(train_dataloader))

In [ ]:
# # Obter um lote de imagens do DataLoader
# batch_images, batch_labels = next(iter(train_dataloader))

# # Plotar as imagens e mostrar seus shapes
# fig, axs = plt.subplots(2, len(batch_images), figsize=(24, 8))
# formato = []
# for i in range(len(batch_images)):
#     formato.append((batch_images[i].shape,batch_labels[i].shape,'\n'))
#     # Converter as imagens e rótulos de tensor para arrays numpy
#     image_np = batch_images[i].numpy().transpose((1, 2, 0))  # Transpor as dimensões para (height, width, channels)
#     label_np = batch_labels[i].numpy().transpose((1, 2, 0))  # Transpor as dimensões para (height, width, channels)

#     # Plotar a imagem original
#     axs[0, i].imshow(image_np)
#     axs[0, i].set_title(f'Image Shape: {image_np.shape}')

#     # Plotar o rótulo
#     axs[1, i].imshow(label_np)
#     axs[1, i].set_title(f'Label Shape: {label_np.shape}')

# # Ajustar espaçamento e exibir o gráfico
# plt.tight_layout()
# plt.show()


In [ ]:
# #print(formato)
# print(f"""
# {batch_labels[0]}

# {batch_images[0]}

# """)

### Testando Num_worker

https://chtalhaanwar.medium.com/pytorch-num-workers-a-tip-for-speedy-training-ed127d825db7

In [ ]:
# from time import time
# import multiprocessing as mp
# for num_workers in range(2, mp.cpu_count(), 2):
#     train_loader = DataLoader(data_test,shuffle=True,num_workers=num_workers,batch_size=8,pin_memory=True)
#     start = time()
#     for epoch in range(1, 3):
#         for i, data in enumerate(train_loader, 0):
#             pass
#     end = time()
#     print("Finish with:{} second, num_workers={}".format(end - start, num_workers))

###exemplo

In [ ]:
# #Exemplo seguido
# data = CustomImageDataset(X, y)

# train_size = int(0.75*len(data))
# idx  = torch.randperm(len(data))
# train_sampler = SubsetRandomSampler(idx[0:train_size])
# test_sampler = SubsetRandomSampler(idx[train_size:])

# train_loader = DataLoader(data, sampler=train_sampler,
#                           batch_size=10, num_workers=2)

# test_loader  = DataLoader(data, sampler=test_sampler,
#                           batch_size=10, num_workers=2)

## Importa DeepLabV3+




In [ ]:
# import network as net
# #é diferente desta maneira que eu fiz ver codigo fonte do git
# model = net.deeplabv3plus_resnet101()
# net.convert_to_separable_conv(model.classifier)
# model = model.to(device)
# #utils.set_bn_momentum(model.backbone, momentum=0.01)
# #model.state_dict()

In [ ]:
#model = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_resnet50', pretrained=True)
# or any of these variants
#model = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_resnet101', pretrained=True,)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_mobilenet_v3_large', pretrained=True)
#model.eval()
#model.to(device)

In [ ]:
from torchvision.models.segmentation import deeplabv3_resnet101 as deeplabv3_resnet101
model = deeplabv3_resnet101(num_classes=2)
#model.to("cpu")
model.to(device)

Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:00<00:00, 287MB/s]


###Setup Loss and Optimizer

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
#torch.optim.lr_scheduler.StepLR(optimizer, step_size= , gamma= )
criterion = nn.CrossEntropyLoss()

##Treino


### Helper functions

In [ ]:
import requests
from pathlib import Path

# Download helper functions from Learn PyTorch repo (if not already downloaded)
if Path("helper_functions.py").is_file():
  print("helper_functions.py already exists, skipping download")
else:
  print("Downloading helper_functions.py")
  # Note: you need the "raw" GitHub URL for this to work
  request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py")
  with open("helper_functions.py", "wb") as f:
    f.write(request.content)



helper_functions.py already exists, skipping download


In [ ]:
# Import accuracy metric
from helper_functions import accuracy_fn # Note: could also use torchmetrics.Accuracy(task = 'multiclass', num_classes=len(class_names)).to(device)

from timeit import default_timer as timer
def print_train_time(start: float,
                     end: float,
                     device: torch.device = None):
  """Prints difference between start and end time. """
  total_time = end-start
  print(f"Train time on {device}: {total_time:.3f} seconds")
  return total_time

In [ ]:
model.state_dict

### Treino

* 21 classes de saida. Talvez terei que doiminuir o numero de classes


In [ ]:
# Set the seed and start the timer
torch.manual_seed(42)
# Import tqdm for progress bar
from tqdm.auto import tqdm


train_time_start_on_cpu = timer()

# Set the number of epochs (we'll keep this small for faster training times)
epochs = 3

# Create training and testing loop
for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}\n-------")
    ### Training
    train_loss = 0
    #loop = tqdm(train_dataloader)
    # Add a loop to loop through training batches # muitos valores para descompactar
    for batch, (X, y) in enumerate(train_dataloader):
        #print(X.shape,y.shape)
        model.train()
        X = X.to(device)
        y = y.squeeze(1).type(torch.long).to(device)
        #y = y.squeeze(1).to(device, dtype=torch.long)
        # 1. Forward pass
        y_pred = model(X)['out']
        # print(f"""

        # Formatos: X: {X.shape}
        #           Y: {y.shape},
        #           Y_pred: {y_pred.shape},
        #           batch: {batch}

        # """)

        # 2. Calculate loss (per batch)
        loss = criterion(y_pred, y)
        train_loss += loss # accumulatively add up the loss per epoch

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Print out how many samples have been seen
        #if batch % 100 == 0:
        print(f"Looked at {batch * len(X)}/{len(train_dataloader.dataset)} samples")

    ## Divide total train loss by length of train dataloader (average loss per batch per epoch)
    #train_loss /= len(train_dataloader)

    ### Testing
    # Setup variables for accumulatively adding up loss and accuracy
    test_loss, test_acc = 0, 0
    model.eval()
    with torch.inference_mode():
        for X, y in test_dataloader:
            X = X.to(device)
            y = y.squeeze(1).type(torch.long).to(device)

            # 1. Forward pass
            test_pred = model(X)['out']

            # 2. Calculate loss (accumatively)
            test_loss += criterion(test_pred, y) # accumulatively add up the loss per epoch

            # 3. Calculate accuracy (preds need to be same as y_true)
            test_acc += accuracy_fn(y_true=y, y_pred=test_pred.argmax(dim=1))

        # Calculations on test metrics need to happen inside torch.inference_mode()
        # Divide total test loss by length of test dataloader (per batch)
        test_loss /= len(test_dataloader)

        # Divide total accuracy by length of test dataloader (per batch)
        test_acc /= len(test_dataloader)

    ## Print out what's happening
    print(f"\nTrain loss: {train_loss:.5f} | Test loss: {test_loss:.5f}, Test acc: {test_acc:.2f}%\n")

# Calculate training time
train_time_end_on_cpu = timer()
total_train_time_model_0 = print_train_time(start=train_time_start_on_cpu,
                                           end=train_time_end_on_cpu,
                                           device=str(next(model.parameters()).device))

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch: 0
-------


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Looked at 0/29 samples
Looked at 8/29 samples
Looked at 16/29 samples

Train loss: 1.75584 | Test loss: 0.68462, Test acc: 4336659.38%

Epoch: 1
-------
Looked at 0/29 samples
Looked at 8/29 samples
Looked at 16/29 samples

Train loss: 1.67982 | Test loss: 0.68035, Test acc: 4016021.88%

Epoch: 2
-------
Looked at 0/29 samples
Looked at 8/29 samples
Looked at 16/29 samples

Train loss: 1.48195 | Test loss: 0.69632, Test acc: 3296003.12%

Train time on cuda:0: 13.137 seconds


Usar 🇰* https://pytorch.org/vision/stable/models/generated/torchvision.models.segmentation.deeplabv3_resnet101.html#torchvision.models.segmentation.deeplabv3_resnet101

a codificacao do target esta errada e proporcao em que ele foi normalizada esta ruim
dividir todos por 255?
normalizar de 0 a 1
deixar de 0 a 255?
utilizar one-hot encode?? (abordagem multiclasse)(mapear os pixels nas classes)

In [ ]:
torch.__version__

In [ ]:
torch.cuda.empty_cache()


In [ ]:
nvidia-smi